In [9]:
from cashe import get_train_test_split, get_testing_inputs, drop_nomodel_columns
from src.data_local import MergedDataPreprocessing, read_last_date, append_last_line
from src.utilities_rejection import DataFrameRejection
from src.lstm_encoder import LSTMEmbedding
from src.model_app import ModelApplicationDeployment

In [10]:
_, df_test = get_train_test_split()

train_rej = DataFrameRejection(df_test)
train_rej_attr = train_rej.map_rejection_label()
df_test  = train_rej.get_label_rejection()

LAST_DATE = read_last_date() ## we should take data released after that day to do the analysis, but we'll take the df_test now
## read data for LAST_DATE
## put it in df_test dataframe

X_test, y_test = get_testing_inputs(df_test)
X_test = drop_nomodel_columns(X_test)

In [11]:
y_test[:4]

,OUTCOME,SUBMIT_CLAIM_MESSAGE,NPHIES_LABEL
6,1,Approved,0
17,0,SUBMITTED AS RELATED TO Inapropriate approval ...,0
18,0,SUBMITTED AS RELATED TO Inapropriate approval ...,1
19,0,SUBMITTED AS RELATED TO Inapropriate approval ...,0


In [12]:
preprocessing_test = MergedDataPreprocessing(X_test)
X_test_prep = preprocessing_test.columns_prep(service_encoding=False)
lstm_embedding = LSTMEmbedding()

X_test_encoded = preprocessing_test.column_embedding(X_test_prep)

E:\Projects\Claims_Rejection\src\data_local.py:275: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df[column] = self.df[column].replace(column_encoding)
E:\Projects\Claims_Rejection\src\data_local.py:280: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df['PatientAgeRange'] = self.df.PatientAgeRange.replace(age_encoding)


In [13]:
model = ModelApplicationDeployment(X_test)

XGBoost Model is loading from disk..


In [14]:
from sklearn.metrics import accuracy_score

pred_claims, pred_rejection = model.predict()

acc1 = accuracy_score(pred_claims.tolist(), y_test['OUTCOME'].tolist())
acc1

0.7650342285413375

In [16]:
### 1 : medically rejected
acc2 = accuracy_score(pred_rejection.tolist(), y_test['NPHIES_LABEL'].tolist())
acc2

0.8818062137967351

## New pipeline

In [1]:
from cashe import get_train_test_split,get_training_inputs, drop_nomodel_columns, get_cashed_input
from src.data_local import MergedDataPreprocessing,read_last_date, append_last_line
from src.lstm_encoder import LSTMEmbedding
from src.model_app import ModelApplication

In [2]:
df_train, df_test = get_train_test_split()

In [3]:
df_train['CREATION_DATE']

index
0        2024-04-04 22:57:57
1        2024-04-04 23:02:21
2        2024-04-04 23:08:38
3        2024-04-04 22:26:08
4        2024-04-04 22:34:22
                 ...        
488828   2024-01-23 17:20:54
488829   2024-01-28 01:01:05
488830   2024-01-28 01:01:05
488831   2024-01-28 01:01:05
488832   2024-01-23 16:35:34
Name: CREATION_DATE, Length: 155399, dtype: datetime64[ns]

In [4]:
append_last_line('2024-04-30')
read_last_date()

'2024-04-30'

In [5]:
X_train, y_train, X_test, y_test = get_training_inputs(df_train,df_test)

In [6]:
X_train = drop_nomodel_columns(X_train)
X_test = drop_nomodel_columns(X_test)
X_train

,NET_WITH_VAT,SERVICE_DESCRIPTION,QTY,LINE_CLAIMED_AMOUNT,LINE_CLAIMED_AMOUNT_SAR,CO_PAY,CO_INSURANCE,LINE_ITEM_DISCOUNT,NET_AMOUNT,NET_VAT_AMOUNT,...,TOTAL_DISCOUNT,TOTAL_DEDUCTIBLE,TOTAL_PATIENT_VATAMOUNT,TOTAL_NET_VAT_AMOUNT,DEPARTMENT_TYPE,TREATMENT_TYPE,PURCHASER_CODE,NEW_BORN,ICD10,PatientAgeRange
index,,,,,,,,,,,,,,,,,,,,,
0,56.12,Unlisted Code,1.0,61.00,61.00,0.00,61.00,12.20,48.80,7.32,...,219.46,139.88,0.0,128.79,18.0,1,2622,0,Z36,6
1,202.40,TSH,1.0,220.00,220.00,0.00,220.00,44.00,176.00,26.40,...,320.25,100.00,0.0,154.46,6.0,1,2622,0,R22.1,5
2,408.08,In vivo laboratory,1.0,475.00,475.00,10.75,464.25,118.75,345.50,51.83,...,390.00,100.00,0.0,180.77,11.0,1,2622,0,R53,6
3,24.27,ELICA-M CREAM 30 GM,1.0,25.55,25.55,4.85,20.70,1.28,19.42,0.00,...,95.07,90.24,0.0,48.30,18.0,1,2622,0,R10,6
4,80.64,Peripheral intravenous cannula,1.0,90.00,90.00,14.40,75.60,18.00,57.60,8.64,...,291.19,103.74,0.0,412.62,39.0,0,2622,0,R00.1,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488828,242.48,Quantitative Troponin (Protein) Analysis,1.0,324.39,324.39,0.00,324.39,113.54,210.85,31.63,...,5229.23,0.00,0.0,5231.53,NaN,0,183,0,I24.9,8
488829,1.06,NEBILET 5MG TABLETS,1.0,1.14,1.14,0.00,1.14,0.08,1.06,0.00,...,13042.16,0.00,0.0,4608.10,NaN,0,183,0,R07.4,7
488830,1.14,Unlisted Procedure Code,5.0,1.77,1.77,0.00,1.77,0.62,1.15,0.00,...,13042.16,0.00,0.0,4608.10,NaN,0,183,0,R07.4,7


In [7]:
preprocessing_train = MergedDataPreprocessing(X_train)
X_train_prep = preprocessing_train.columns_prep(service_encoding=False)

preprocessing_test = MergedDataPreprocessing(X_test)
X_test_prep = preprocessing_test.columns_prep(service_encoding=False)

C:\AI_Projects\Claims_Rejection_2024\Claims_Rejection\src\data_local.py:280: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df['PatientAgeRange'] = self.df.PatientAgeRange.replace(age_encoding)


In [8]:
X_train_prep

,NET_WITH_VAT,SERVICE_DESCRIPTION,QTY,LINE_CLAIMED_AMOUNT,LINE_CLAIMED_AMOUNT_SAR,CO_PAY,CO_INSURANCE,LINE_ITEM_DISCOUNT,NET_AMOUNT,NET_VAT_AMOUNT,...,TOTAL_DISCOUNT,TOTAL_DEDUCTIBLE,TOTAL_PATIENT_VATAMOUNT,TOTAL_NET_VAT_AMOUNT,DEPARTMENT_TYPE,TREATMENT_TYPE,PURCHASER_CODE,NEW_BORN,ICD10,PatientAgeRange
index,,,,,,,,,,,,,,,,,,,,,
0,56.12,Unlisted Code,1.0,61.00,61.00,0.00,61.00,12.20,48.80,7.32,...,219.46,139.88,0.0,128.79,18.0,1,2622,0,22,6
1,202.40,TSH,1.0,220.00,220.00,0.00,220.00,44.00,176.00,26.40,...,320.25,100.00,0.0,154.46,6.0,1,2622,0,19,5
2,408.08,In vivo laboratory,1.0,475.00,475.00,10.75,464.25,118.75,345.50,51.83,...,390.00,100.00,0.0,180.77,11.0,1,2622,0,19,6
3,24.27,ELICA-M CREAM 30 GM,1.0,25.55,25.55,4.85,20.70,1.28,19.42,0.00,...,95.07,90.24,0.0,48.30,18.0,1,2622,0,19,6
4,80.64,Peripheral intravenous cannula,1.0,90.00,90.00,14.40,75.60,18.00,57.60,8.64,...,291.19,103.74,0.0,412.62,39.0,0,2622,0,19,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488828,242.48,Quantitative Troponin (Protein) Analysis,1.0,324.39,324.39,0.00,324.39,113.54,210.85,31.63,...,5229.23,0.00,0.0,5231.53,NaN,0,183,0,10,8
488829,1.06,NEBILET 5MG TABLETS,1.0,1.14,1.14,0.00,1.14,0.08,1.06,0.00,...,13042.16,0.00,0.0,4608.10,NaN,0,183,0,19,7
488830,1.14,Unlisted Procedure Code,5.0,1.77,1.77,0.00,1.77,0.62,1.15,0.00,...,13042.16,0.00,0.0,4608.10,NaN,0,183,0,19,7


In [9]:
lstm_embedding = LSTMEmbedding()

X_train_encoded = preprocessing_train.column_embedding(X_train_prep)
X_test_encoded = preprocessing_train.column_embedding(X_test_prep)

In [10]:
X_test_encoded

,NET_WITH_VAT,QTY,LINE_CLAIMED_AMOUNT,LINE_CLAIMED_AMOUNT_SAR,CO_PAY,CO_INSURANCE,LINE_ITEM_DISCOUNT,NET_AMOUNT,NET_VAT_AMOUNT,PATIENT_VAT_AMOUNT,...,CombinedText7,CombinedText8,CombinedText9,CombinedText10,CombinedText11,CombinedText12,CombinedText13,CombinedText14,CombinedText15,CombinedText16
index,,,,,,,,,,,,,,,,,,,,,
9,27.36,2.0,28.80,28.80,5.47,23.33,1.44,21.89,0.00,0.0,...,285,54,6,70,36,43,73,22,0,0
26,12.16,20.0,13.15,13.15,6.08,7.07,0.99,6.08,0.00,0.0,...,0,0,0,0,0,0,0,0,0,0
27,50.18,10.0,54.26,54.26,10.04,44.22,4.07,40.15,0.00,0.0,...,0,0,0,0,0,0,0,0,0,0
28,27.47,3.0,29.70,29.70,5.49,24.21,2.23,21.98,0.00,0.0,...,0,0,0,0,0,0,0,0,0,0
69,131.26,1.0,163.06,163.06,0.00,163.06,48.92,114.14,17.12,0.0,...,19,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488709,836.67,1.0,1053.50,1053.50,76.00,977.50,316.05,661.45,99.22,11.4,...,617,32,205,703,172,37,200,441,0,0
488710,134.40,1.0,120.00,120.00,24.00,96.00,0.00,96.00,14.40,3.6,...,0,0,0,0,0,0,0,0,0,0
488775,12.49,20.0,13.15,13.15,2.50,10.65,0.66,9.99,0.00,0.0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
model_runner = ModelApplication(X_train= X_train,y_train= y_train['OUTCOME'],X_test= X_test,y_test= y_test['OUTCOME'])

In [12]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 155399 entries, 0 to 488832
Data columns (total 97 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   CREATED_BY_USER              155399 non-null  int64         
 1   CREATION_DATE                155399 non-null  datetime64[ns]
 2   VISIT_ID                     155399 non-null  int64         
 3   NET_WITH_VAT                 155399 non-null  float64       
 4   SERVICE_DESCRIPTION          155399 non-null  object        
 5   QTY                          155399 non-null  float64       
 6   LINE_CLAIMED_AMOUNT          155399 non-null  float64       
 7   LINE_CLAIMED_AMOUNT_SAR      155399 non-null  float64       
 8   CO_PAY                       155399 non-null  float64       
 9   CO_INSURANCE                 155399 non-null  float64       
 10  LINE_ITEM_DISCOUNT           155399 non-null  float64       
 11  NET_AMOUNT                   15

In [13]:
model_runner.get_model(force_retrain=True)

Training is done


In [14]:
model_runner.evaluate_model()

{'XGBoost': {'Accuracy': 0.75,
  'Precision': 0.74,
  'Recall': 0.79,
  'F1 Score': 0.77}}

In [15]:
y_train

,OUTCOME,SUBMIT_CLAIM_MESSAGE
index,,
0,0,BE-1-1 Co-pay was not collected from memberBE-...
1,0,BE-1-7 Incorrect billing regimePartial Approved
2,1,BE-1-7 Incorrect billing regimePartial Approved
3,1,MN-1-1 Service is not clinically justified bas...
4,0,BE-1-7 Incorrect billing regimeBE-1-7 Incorrec...
...,...,...
488828,0,BE-1-4 Preauthorization is required and was no...
488829,1,MN-1-1 Service is not clinically justified bas...
488830,1,MN-1-1 Service is not clinically justified bas...


In [16]:
from src.rejection_clustering import RejectionCategory
from src.rejection_clustering import find_category_similarity
rejection_category = RejectionCategory()

messages  = y_test['SUBMIT_CLAIM_MESSAGE'].tolist()
predicted_labels = y_test['OUTCOME'].tolist()

ModuleNotFoundError: No module named 'AHBS_AIServices'

In [33]:
cleaned_messages = rejection_category.preprocess_messages(messages,predicted_labels)

In [45]:
cleaned_messages = set(cleaned_messages)
cleaned_messages

{'2 Bipolar affective disorder Not Covered',
 '2 Contraceptive management Not Covered',
 '2 Contraceptive management unspecified Not Covered',
 '2 Convalescence following surgery Not Covered',
 '2 F U care r o fx plate oth int fix dev Not Covered',
 '2 F U exam after oth Rx for cond Not Covered',
 '2 Female infertility Not Covered',
 '2 Follow up exam after surg for oth cond Not Covered',
 '2 General medical examination Not Covered',
 '2 Impotence of organic origin Not Covered',
 '2 Obesity unspecified Not Covered',
 '2 Obs suspected toxic effect ingested subs Not Covered',
 '2 Patent ductus arteriosus Not Covered',
 '2 Postinflammatory hyperpigmentation Not Covered',
 '2 This Diagnosis is not Covered',
 '2 This Diagnosis is not covered',
 '2 This diagnosis not coverd',
 '2024 284841 missing discharge summary progress notes',
 'AD-1-3 Diagnosis is inconsistent with encounter type',
 'AD-1-4 Diagnosis is inconsistent with service/procedure',
 'AD-1-6 Service/procedure is inconsistent wi

In [46]:
_,classification_result = find_category_similarity(cleaned_messages)

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
#!pip install git+https://github.com/Andalusia-Data-Science-Team/AHBS_AIServices

In [47]:
categories = [ "Medical Justification Denials",  "Contractual or Financial Benefit Denials", "Information Errors or Omissions Denials"]

sim_out = similarity.sentences_cosine_similarity(list(set(cleaned_messages)), list(categories))

AttributeError: 'list' object has no attribute 'shape'

## Data Update

In [8]:
from src.data_query import  update_data
from src.data_local import read_last_date, append_last_line , create_folder,  store_dfs

In [9]:
LAST_DATE = read_last_date()
df_visit, df_service,df_diagnose = update_data(LAST_DATE)

In [10]:
LAST_DATE

'2024-05-31'

In [11]:
NEW_LAST_DATE = str(df_visit['CREATION_DATE'].sort_values(ascending=False).iloc[0])[:10]
path_date = 'data/HJH/updates/' + NEW_LAST_DATE
create_folder(folder_name= path_date)
store_dfs(df_visit, df_service,df_diagnose,path_date)
path_date

'data/HJH/updates/2024-05-31'

In [20]:
df_visit.CREATION_DATE.min()

Timestamp('2024-01-01 09:58:15')

In [12]:
append_last_line(NEW_LAST_DATE)

In [15]:
df_visit['CREATION_DATE'].min()

Timestamp('2024-01-01 09:58:15')

In [22]:
df_visit['CREATION_DATE'].dt.month.unique()

array([1, 2, 3, 4, 5])